In [2]:
import numpy as np
import scipy
import scipy.sparse
import scipy.sparse.linalg
import glob
import sys
import threading
import queue
import pathlib


In [3]:
def file_to_differential(path):
    vec = ""
    shape = ""
    with open(path, 'r') as reader:
        s = reader.readline().split(" ")
        shape = (int(s[0]), int(s[1]))

        vec = reader.readline()[10:-3]
    
    rows = []
    cols = []
    vals = []
    
    bitmask = np.int64(2**32)
    
    for elem in vec.split(", "):
        n = np.int64(elem)
        rows.append((abs(n) // bitmask) - 1)
        cols.append((abs(n) % bitmask) - 1)
        vals.append(1 if n > 0 else -1)
    
    return (vals, (rows, cols)), shape

In [4]:


def get_lap_eigs(laplacian):
    # get smallest eigenpair

    eig_val_small = -1
    # if laplacian.det() != 0: # @TODO: somehow decide if this laplacian is nonsingular
    # eig_val_small, eig_vec_small = scipy.sparse.linalg.eigsh(laplacian, k=1, sigma=0, maxiter=1000)
    eig_vals = scipy.linalg.eigh(laplacian.toarray(), eigvals_only=True)
    
    # get only largest eigenvalue using sparse matrix
    # eig_val_big, eig_vec_big = scipy.sparse.linalg.eigsh(laplacian, k=1, maxiter=1000)

    # return (eig_val_small, eig_val_big)
    return eig_vals



In [5]:
def read_differential_files(dir):
    files = glob.glob(dir + "/*")

    dict = {}

    for file in files:
        data = file.split("_")
        i = int(data[-2])
        j = int(data[-1])
        
        dict[(i,j)] = file
    
    return dict

In [8]:
# compute the dimension of the homology indicates the number of eigenvalues of laplacian
# that are exactly zero
def get_num_zero_eigs(d_i_minus_1, d_i):
    img_dim = np.linalg.matrix_rank(d_i_minus_1.to_array())
    ker_dim = d_i.shape[1] - np.linalg.matrix_rank(d_i.to_array())
    return ker_dim - img_dim

get_num_zero_eigs(get_diff_matrix())

In [6]:

def get_diff_matrix(path):
    (vals, (rows, cols)), s = file_to_differential(path)
    return scipy.sparse.coo_matrix((vals, (rows, cols)), shape=s).asfptype()

def write_laplacian_sparsity(laplacian, crossings, index, i, j):
    sparsity = laplacian.nnz

    with open("./laplacian_sparsity/knot_" + str(crossings) + "_" + str(index) + "_laspa", "a+") as writer:
        writer.write(str(i) + " " + str(j) + " " + str(laplacian.nnz) + " " + str(laplacian.shape[0]) + "\n")

def get_knot_eigs(dir, crossings, index):

    # @TODO: pass crossings and index to read_differential instead of parsing
    dict = read_differential_files(dir)
    keys = dict.keys()

    laplacians = {}

    for (i, j) in sorted(keys, reverse=True):

        d_i = get_diff_matrix(dict[(i,j)])

        laplacian = 0
        if (i-1,j) in keys:
            d_i_minus_1 = get_diff_matrix(dict[(i-1,j)])
            laplacian = (d_i_minus_1 * d_i_minus_1.transpose()) + (d_i.transpose() * d_i)         
            laplacians[(i,j)] = laplacian

        else:
            laplacian = d_i.transpose() * d_i    
            laplacians[(i,j)] = laplacian            

        # @TODO: save laplacian sparsity
        
        if (i+1,j) not in laplacians:
            laplacian = d_i * d_i.transpose()
            laplacians[i+1,j] = laplacian
    
    with open("./eigs/knot_" + str(crossings) + "_" + str(index) + "_eigs", "w+") as writer:
        for (i,j) in laplacians.keys():
            write_laplacian_sparsity(laplacians[(i,j)], crossings, index, i, j)
        
            # if laplacians[(i,j)].shape == (1,1):
            #     continue

            # eig_small, eig_large = get_eigs(laplacians[(i,j)])
            eig_vals = get_lap_eigs(laplacians[(i,j)])

            # write eigenvalues to file
            output_line = str(i) + " " + str(j) + " "
            writer.write(output_line)
            for e in eig_vals:
                writer.write(str(e) + " ")
            writer.write("\n")

get_knot_eigs("./KhoHo/differentials/knot_5_1/", 5, 1)


In [7]:

q = queue.Queue()

def worker():
    while (True):
        task = q.get()
        print("working on " + str(task["crossings"]) + "_" + str(task["index"]))
        get_knot_eigs(task["path"], task["crossings"], task["index"])
        
        q.task_done()

MAX_THREAD_NUM = 5
threads = [threading.Thread(target=worker, daemon=True).start() for i in range(MAX_THREAD_NUM)]

index_count = [1, 1, 2, 3, 7, 21, 49, 165]
for crossings in range(3,11):
    for index in range(1,index_count[crossings - 3] + 1):
        path = "./KhoHo/differentials/knot_" + str(crossings) + "_" + str(index)
        dir = pathlib.Path(path)
        
        if dir.is_dir():
            q.put({"path":path, "crossings": crossings, "index": index})

q.join()


working on 3_1working on 4_1working on 5_1working on 5_2



working on 6_1
working on 6_2
working on 6_3
working on 7_1
working on 7_2
working on 7_3
working on 7_4working on 7_5
working on 7_6

working on 7_7
working on 8_1
working on 8_2
working on 8_3
working on 8_4
working on 8_5
working on 8_6working on 8_7

working on 8_8
working on 8_9working on 8_10

working on 8_11working on 8_12

working on 8_13
working on 8_14working on 8_15

working on 8_16
working on 8_17
working on 8_18working on 8_19

working on 8_20
working on 8_21
working on 9_1working on 9_2
working on 9_3
working on 9_4

working on 9_5
working on 9_6
working on 9_7
working on 9_8working on 9_9

working on 9_10
working on 9_11
working on 9_12working on 9_13

working on 9_14
working on 9_15
working on 9_16
working on 9_17
working on 9_18
working on 9_19working on 9_20working on 9_21


working on 9_22
working on 9_23
working on 9_24
working on 9_25working on 9_26

working on 9_27
working on 9_28
working on 9_29
working 

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-7-75431262c07e>", line 7, in worker
    get_knot_eigs(task["path"], task["crossings"], task["index"])
  File "<ipython-input-6-a6908ebfe624>", line 53, in get_knot_eigs
    write_laplacian_sparsity(laplacians[(i,j)], crossings, index, i, j)
  File "<ipython-input-6-a6908ebfe624>", line 8, in write_laplacian_sparsity
    with open("./laplacian_sparsity/knot_" + str(crossings) + "_" + str(index) + "_laspa", "a+") as writer:
FileNotFoundError: [Errno 2] No such file or directory: './laplacian_sparsity/knot_10_1_laspa'



KeyboardInterrupt: 

In [ ]:
def get_file_name(crossings, index, type):
    if type == "differential":
        return "knot_" + str(crossings) + "_" + str(index)
    if type == "eig":
        return "knot_" + str(crossings) + "_" + str(index) + "_eigs"
    if type == "laplacian":
        return "knot_" + str(crossings) + "_" + str(index) + "_laspa"
